In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import os
from datetime import datetime

def file_config(start_url, i=2):
    # Configuration du driver
    driver_path = r"/usr/local/bin/chromedriver"
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--headless')
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    driver.set_page_load_timeout(6000)
    attempts = i
    while attempts >0:
        try :
            driver.get(start_url)
            driver.maximize_window()
            attempts = 0
        except TimeoutException:
            attempts = attempts - 1
    return driver

driver = file_config(start_url = r'https://www.ha-properties.com/location/appartement?prod.prod_type=appt&displayType=gallery')

wait = WebDriverWait(driver, 30)

data = list()

def notif_check():
    wait = WebDriverWait(driver, 5)
    try:
        non_merci_button = wait.until(EC.visibility_of_element_located((By.XPATH, '#root > div._vtjlnt._3hmsj._1ymchky.theme1 > div._d5sco8 > div._mxwtc7')))
        non_merci_button.click()
    except:
        pass

In [ ]:
data = []
i = 1

In [ ]:
def scrape_data(cards):

    notif_check()
    links = []
    for i in range(len(cards)):
        link = cards[i].get_attribute('href')
        links.append(link)

    for link in links:
        attempts = 3
        while attempts >0:
            try :
                driver.get(link)
                driver.refresh()
                attempts = 0
            except TimeoutException:
                attempts = attempts - 1

        notif_check()
        try:
            # Utilisation de BeautifulSoup pour récupérer les détails de l'annonce
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            try :
                details = soup.select_one('#root > main')
            except AttributeError:
                continue

            try:
                # from details_group1
                price = details.select_one(r"#\33 3034c87-4ccc-4141-9386-bf9bc1e794f6 > div > div > div > p").text.strip()
                title = details.select_one(r"#\34 edce9b1-5968-4982-bbde-63950368f746 > div > div > div > h1").text.strip()
            except AttributeError:
                price = None
                title = None

            localisation = None
            immo_type = "Appartements"


            # from details_group2
            try:
                superficie = details.select_one(r'#\35 7612fdc-e6b1-4636-ac10-34cadc534746 > div > div > div > ul > li:nth-child(1) > div > div._1qeaafx > span._1acok99._xajb81.textblock').text.strip()
                nb_pieces = details.select_one(r'#\35 7612fdc-e6b1-4636-ac10-34cadc534746 > div > div > div > ul > li:nth-child(2) > div > div._1qeaafx > span._1acok99._xajb81.textblock').text.strip()
                nb_salle_de_bain = details.select_one(r'#\35 7612fdc-e6b1-4636-ac10-34cadc534746 > div > div > div > ul > li:nth-child(3) > div > div._1qeaafx > span._1acok99._xajb81.textblock').text.strip()
            except AttributeError:
                superficie = None
                nb_pieces = None
                nb_salle_de_bain = None

            # from details_group3
            try:
                description = details.select_one('div:nth-child(5) > div > div > div > div > div._wqiuz2 > div > div').text.strip().replace('\n', ' ')
            except AttributeError:
                description = None


            annonceur = "H_and_A"

            current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            data.append({
                'title': title,
                'price': price,
                'localisation': localisation,
                'superficie': superficie,
                "type d'immobilier": immo_type,
                'nb_pieces': nb_pieces,
                "nb_salle_de_bain": nb_salle_de_bain,
                'scraping_date': current_datetime,
                "annonceur" : annonceur,
                "link" : link,
                'description': description
            })

            print(data[-1])
        except :
            continue

    return data


In [ ]:
while True:

    try :
        # Attendre que les éléments soient visibles
        cards_presence = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#ece4f03b-f1f7-4ba3-872f-524ae1671efe > div > div > div._11upq6o')))
        # Créer une liste de cartes
        cards = cards_presence.find_elements(By.CSS_SELECTOR, 'a._gozzbg')
        
        if len(data) == 1000:
            break
        scrape_data(cards)
        # save_data(data)
        i += 1
        driver.get(f'https://www.ha-properties.com/location/appartement?prod.prod_type=appt&displayType=gallery&page={i}')
        driver.refresh()
        print(data)
    except :
        print("Une erreur c'est produite lors de la collecte sur H_and_A_Maisons, ou derniere page atteinte")
        break


print("fin du scraping")
driver.quit()

In [ ]:
data = pd.DataFrame(data)
data

import locale
# Définir la locale en français
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

# Obtenir la date et l'heure actuelles
current_datetime = datetime.now()
formatted_date = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
formatted_month = current_datetime.strftime("%B")
formatted_day = current_datetime.strftime("%d")


import platform
if platform.system() == 'Windows':
    dynamic_path = f'D:\\Bureau\\MemoiresStages\\Travaux_techniques\\Scrapping\\Datasets\\{formatted_month}\\{formatted_day}_{formatted_month}\\H_and_A_Maisons_{formatted_day}_{formatted_month}.csv'
else:
    dynamic_path = f'/mnt/d/Bureau/MemoiresStages/Travaux_techniques/Scrapping/Datasets/{formatted_month}/{formatted_day}_{formatted_month}/H_and_A_Maisons_{formatted_day}_{formatted_month}.csv'

os.makedirs(os.path.dirname(dynamic_path), exist_ok=True)

data.to_csv(dynamic_path, index=False)